In [2]:
import json
with open('./config/config_test_small_local.json') as config_file:
    config = json.load(config_file)
    
# print(config)    
collection_key = "de70"    
es_host = config["collections_info_elastic"][collection_key]["es_host"]
es_port = config["collections_info_elastic"][collection_key]["es_port"]
es_index = config["collections_info_elastic"][collection_key]["es_index"]
print(es_index, es_host, es_port)

de70 localhost 9292


In [83]:
from datetime import datetime
from elasticsearch import Elasticsearch
import json


class Documentdb:

    def __init__(self, el_host, el_port):

        try:
            self.es = Elasticsearch([{'host': el_host, 'port': el_port}],
                                   timeout=30, max_retries=3, retry_on_timeout=True)
        except:
            print("in documentdb init exception occured in es")

    def search(self, jo, bim, body, es_index="corona_news"):
        print(body)
        self.es.indices.refresh(index=es_index)
        res = self.es.search(index=es_index,
                             body=body)
        
        # print(res)
        return res

documentdb = Documentdb(es_host, es_port)


In [103]:
body ={
     "size":10000,
     "query": {
         "nested": {
             "path": "jobim",
             "query": {
                 "bool": {
                     "must": [
                         {"match": {"jobim.jo": jo}},
                         {"match": {"jobim.bim": bim}}
                     ]
                 }
             }
         }
     }

 }

body_single={
    "size" : 10000,
    "query": {
    "bool": {
      "must": [
        {
          "nested": {
            "path": "jobim",
            "query": {
                 "bool": {
                     "must": [
                         {"match": {"jobim.jo": jo}},
                         {"match": {"jobim.bim": bim}},
                     ]
                 }
             } 
          }
        },
        {
          "match": {
            "time_slice": time_slice
          }
        }
      ]
    }
  }
 }


body_multiple={
    "size" : 10000,
    "query": {
    "bool": {
      "must": [
        {
          "nested": {
            "path": "jobim",
            "query": {
                 "bool": {
                     "must": [
                         {"match": {"jobim.jo": jo}},
                         {"match": {"jobim.bim": bim}},
                     ]
                 }
             } 
          }
        },
        {
          "bool": {
              "should":[{"match":{"time_slice":time_slice}} for time_slice in time_slices]

            }
        }
      ]
    }
  }
 }



In [102]:
time_slices = ['1995-1999']

[{"match":{"time_slice":time_slice}} for time_slice in time_slices]


[{'match': {'time_slice': '1995-1999'}}]

In [88]:
data = {'jo': 'Krise#NN', 'bim': 'von#APPR#-NK', 'time_slice':'2000-2002',
        'collection_key': collection_key}
data = {'jo': 'Krise#NN', 'bim': 'von#APPR#-NK', 'time_slice':'1995-1999',
        'collection_key': collection_key}
data = {'jo': 'Krise#NN', 'bim': 'von#APPR#-NK', 'time_slice':'2003-2006',
        'collection_key': collection_key}


In [104]:
jo = data["jo"]
bim = data["bim"]
time_slice = data["time_slice"]

ret = []
res = documentdb.search(jo, bim, body_multiple, es_index)
ret_set = set()
print("hits:"+str(len(res["hits"]["hits"])))
# print(res)
dates = set()
for hit in res["hits"]["hits"]:
#     print(hit)
    text = hit["_source"]["date"][:10] + ": " \
           + hit["_source"]["sentence"] + \
           " [" + hit["_source"]["source"] + "] "
    ret_set.add(text)
    dates.add(hit["_source"]["date"][:10])

ret_list = list(ret_set)
ret_list.sort()
# print(ret_list)
if len(ret_list) > 0:
    for text in ret_list:
        ret.append({"doc": text})
else:
    ret.append({"doc": "No Results."})


# print({"docs": ret})
print("dates:", dates)

{'size': 10000, 'query': {'bool': {'must': [{'nested': {'path': 'jobim', 'query': {'bool': {'must': [{'match': {'jobim.jo': 'Krise#NN'}}, {'match': {'jobim.bim': 'von#APPR#-NK'}}]}}}}, {'bool': {'should': [{'match': {'time_slice': '1995-1999'}}]}}]}}}
hits:484
dates: {'1995-1999'}


In [ ]:
{'_index': 'de70', '_type': '_doc', '_id': '12292050', '_score': 14.173796, 
 '_source': {'source': 'DIE WELT 2000', 
             'sentence': 'Das Wort von der Krise in der Kunst wird laut. Wie sehen Sie das?', 
             'date': '2000-2002', 'time_slice': '2000-2002', 
             'jobim': [{'jo': 'Wort#NN', 'bim': 'der#ART#NK'}, {'jo': 'Wort#NN', 'bim': 'von#APPR#PG'}, 
                       {'jo': 'Wort#NN', 'bim': 'werden#VA#-SB'}, {'jo': 'Wort#NN', 'bim': 'sehen#VV#-SB'}, 
                       {'jo': 'von#APPR', 'bim': 'Wort#NN#-PG'}, {'jo': 'von#APPR', 'bim': 'Krise#NN#NK'}, 
                       {'jo': 'der#ART', 'bim': 'Krise#NN#-NK'}, {'jo': 'Krise#NN', 'bim': 'der#ART#NK'}, 
                       {'jo': 'Krise#NN', 'bim': 'von#APPR#-NK'}, {'jo': 'Krise#NN', 'bim': 'in#APPR#MNR'}, 
                       {'jo': 'in#APPR', 'bim': 'Krise#NN#-MNR'}, {'jo': 'in#APPR', 'bim': 'Kunst#NN#NK'}, 
                       {'jo': 'der#ART', 'bim': 'Kunst#NN#-NK'}, {'jo': 'Kunst#NN', 'bim': 'der#ART#NK'}, 
                       {'jo': 'Kunst#NN', 'bim': 'in#APPR#-NK'}, {'jo': 'werden#VA', 'bim': 'Wort#NN#SB'}, 
                       {'jo': 'werden#VA', 'bim': 'laut#ADJD#PD'}, {'jo': 'werden#VA', 'bim': 'sehen#VV#CJ'}, 
                       {'jo': 'laut#ADJD', 'bim': 'werden#VA#-PD'}, {'jo': 'laut#ADJD', 'bim': '--#$.#--'}, 
                       {'jo': '--#$.', 'bim': 'laut#ADJD#---'}, {'jo': 'wie#PWAV', 'bim': 'sehen#VV#-MO'}, 
                       {'jo': 'sehen#VV', 'bim': 'Wort#NN#SB'}, {'jo': 'sehen#VV', 'bim': 'der#PDS#OA'}, 
                       {'jo': 'sehen#VV', 'bim': 'sie#PPER#SB'}, {'jo': 'sehen#VV', 'bim': 'werden#VA#-CJ'}, 
                       {'jo': 'sehen#VV', 'bim': 'wie#PWAV#MO'}, {'jo': 'sie#PPER', 'bim': 'sehen#VV#-SB'}, 
                       {'jo': 'der#PDS', 'bim': 'sehen#VV#-OA'}, {'jo': 'der#PDS', 'bim': '--#$.#--'}, 
                       {'jo': '--#$.', 'bim': 'der#PDS#---'}]}}